In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import math
import glob
import pickle
import statistics
import scipy.stats as stats
from sklearn.neighbors import KernelDensity
import dask
import seaborn as sns
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [2]:
def get_files():
    models = glob.glob("/terra/data/cmip5/global/rcp85/*")
    avail={}
    for model in models:
        zg = glob.glob(str(model)+"/r1i1p1/day/2deg/zg*")
        try:
            test = zg[0]
            avail[model.split('/')[-1]] = zg
        except:
             pass
    return avail

In [3]:
files = get_files()

In [4]:
def contourise(x):
    x = x.fillna(0)
    x = x.where((x>=limit))
    x = x/x
    return x

In [5]:
rcp85={}
for model in files:
    print(model)
    x = xr.open_mfdataset(files[model])
    x = x.sel(plev=85000)
    x = x.sel(time=slice('2045','2100'))
    x = x.load()
    x = x.sel(lat=slice(-60,0))
    x = x[['zg']]
    x = x.assign_coords(lon=(((x.lon + 180) % 360) - 180))
    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
        x = x.sortby(x.lon)
    x = x.sel(lon=slice(-50,20))
    x = x.resample(time="QS-DEC").mean(dim="time",skipna=True)
    x = x.load()
    limit = np.nanquantile(x.zg.values,0.9)
    print(limit)
    rcp85[model]={}
    for seas in ['DJF','MAM','JJA','SON']:
        mean_seas = x.where(x.time.dt.season==str(seas)).dropna(dim='time', how = 'all')
        mean_seas = contourise(mean_seas).zg.fillna(0).mean(dim='time')
        rcp85[model][seas] = mean_seas.fillna(0)
    x.close()

FGOALS-g2
1561.8729736328125
ACCESS1-0
1552.7299072265625
CMCC-CM
1554.4324340820312
bcc-csm1-1
1569.9281005859375
CMCC-CMS
1559.1117431640625
CanESM2
1560.0959838867188
GFDL-CM3
1571.4184814453124
BNU-ESM
1573.8772216796874
GFDL-ESM2G
1554.6328247070312
CMCC-CESM
1563.514111328125
MRI-CGCM3
1557.4382690429688
IPSL-CM5A-LR
1548.9581787109375
CNRM-CM5
1549.2410034179688
ACCESS1-3
1560.8900634765625
GFDL-ESM2M
1554.944091796875
MPI-ESM-MR
1560.2386474609375
IPSL-CM5A-MR
1552.9481201171875
HadGEM2-CC
1555.4704833984374
bcc-csm1-1-m
1577.1167114257812
MIROC5
1562.0147705078125
MRI-ESM1
1558.505126953125
IPSL-CM5B-LR
1536.89365234375
MIROC-ESM-CHEM
1573.0644653320312
HadGEM2-ES
1556.4935913085938
MPI-ESM-LR
1558.2178588867187
MIROC-ESM
1562.4359130859375


In [6]:
pickle.dump(rcp85, open( "rcp85_dic.p", "wb" ) )

In [7]:
def get_files():
    models = glob.glob("/terra/data/cmip5/global/historical/*")
    avail={}
    for model in models:
        zg = glob.glob(str(model)+"/r1i1p1/day/2deg/zg*")
        try:
            test = zg[0]
            avail[model.split('/')[-1]] = zg
        except:
             pass
    return avail

In [8]:
files = get_files()

In [9]:
historical={}
for model in files.keys():
    print(model)
    x = xr.open_mfdataset(files[model])
    x = x.sel(plev=85000)
    x = x.sel(time=slice('1950','2005'))
    x = x.load()
    x = x.sel(lat=slice(-60,0))
    x = x[['zg']]
    x = x.assign_coords(lon=(((x.lon + 180) % 360) - 180))
    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
        x = x.sortby(x.lon)
    x = x.sel(lon=slice(-50,20))
    x = x.resample(time="QS-DEC").mean(dim="time",skipna=True)
    x = x.load()
    limit = np.nanquantile(x.zg.values,0.9)
    historical[model]={}
    for seas in ['DJF','MAM','JJA','SON']:
        mean_seas = x.where(x.time.dt.season==str(seas)).dropna(dim='time', how = 'all')
        mean_seas = contourise(mean_seas).zg.fillna(0).mean(dim='time')
        historical[model][seas] = mean_seas.fillna(0)
    x.close()

IPSL-CM5A-LR
MPI-ESM-MR
MRI-ESM1
MIROC-ESM-CHEM
EC-EARTH
HadGEM2-CC
bcc-csm1-1-m
FGOALS-g2
ACCESS1-0
MIROC-ESM
GFDL-CM3
CanESM2
IPSL-CM5B-LR
MPI-ESM-LR
CMCC-CM
GFDL-ESM2G
MIROC5
MPI-ESM-P


/home/pmarsh/.local/lib/python3.8/site-packages/xarray/coding/times.py:119: SerializationWarning: Ambiguous reference date string: 850-1-1 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0850-1-1 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


CMCC-CMS
CNRM-CM5
IPSL-CM5A-MR
ACCESS1-3
MRI-CGCM3
bcc-csm1-1
CMCC-CESM
HadGEM2-AO
BNU-ESM
NorESM1-M
HadCM3
GFDL-ESM2M


In [10]:
pickle.dump(historical, open( "hist_dic.p", "wb" ) )